In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import os
import tarfile
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import cv2

tf.config.run_functions_eagerly(True)
# Extract the dataset
dataset_path = '/kaggle/input/final-project-soc-23b2507/CUB_200_2011.tgz'
extract_path = '/kaggle/extracted/CUB_200_2011'

with tarfile.open(dataset_path, 'r:gz') as tar:
    tar.extractall(extract_path)

# Load the data
images_path = os.path.join(extract_path, 'CUB_200_2011', 'images')
images_list = []
labels_list = []

with open(os.path.join(extract_path, 'CUB_200_2011', 'images.txt')) as f:
    for line in f:
        images_list.append(line.strip().split(' ')[1])

with open(os.path.join(extract_path, 'CUB_200_2011', 'image_class_labels.txt')) as f:
    for line in f:
        labels_list.append(int(line.strip().split(' ')[1]) - 1)


def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128, 128))
    img = img / 255.0
    return img

X = np.array([load_and_preprocess_image(os.path.join(images_path, img_path)) for img_path in images_list])
y = to_categorical(labels_list, num_classes=200)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3, 3), activation='relu')) 
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
#     model.add(layers.Conv2D(256, (3, 3), activation='relu'))  # Additional Conv2D layer
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(200, activation='softmax'))
    
    return model

model = create_cnn_model()
model.summary()


# assert model.count_params() < 10_000_000, "The model exceeds 10 million parameters"


model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),  
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


datagen = ImageDataGenerator(
    rotation_range=40,  
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,  
    zoom_range=0.3, 
    horizontal_flip=True,
    fill_mode='nearest')

batch_size = 64  
epochs = 50  

history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) // batch_size,
                    epochs=epochs,
                    validation_data=(X_test, y_test))

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)              │ (None, 126, 126, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 63, 63, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 61, 61, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 59, 59, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 29, 29, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 27, 27, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 25, 25, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 10, 10, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1024)           │     8,389,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 200)            │       205,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,243,080 (50.52 MB)

 Trainable params: 13,243,080 (50.52 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
118/147 ━━━━━━━━━━━━━━━━━━━━ 8s 290ms/step - accuracy: 0.0034 - loss: 5.2994